# Lab 11: Transformers - Práctica

## Objetivos

En este laboratorio aprenderás:

1. **Self-Attention**: Implementación paso a paso del mecanismo de atención
2. **Multi-Head Attention**: Múltiples cabezas de atención en paralelo
3. **Positional Encoding**: Cómo añadir información de posición
4. **Transformer Blocks**: Construcción de bloques encoder y decoder
5. **BERT Fine-tuning**: Adaptar BERT para análisis de sentimiento
6. **GPT-2 Generation**: Generación de texto con GPT-2
7. **Attention Visualization**: Visualizar y entender patrones de atención
8. **Comparación con RNNs**: Ventajas de Transformers

---

## Setup e Importaciones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Importar implementaciones
import sys
sys.path.append('./codigo')
from transformers import (
    SelfAttentionNumPy, MultiHeadAttentionNumPy,
    PositionalEncodingSinusoidal, create_causal_mask,
    visualize_attention_weights, visualize_multi_head_attention,
    visualize_positional_encoding
)

print("✓ Imports completados")

---
## Parte 1: Self-Attention Paso a Paso

### 1.1 Concepto de Self-Attention

Self-attention permite que cada palabra "atienda" a todas las demás para determinar su representación contextual.

**Fórmula:**
```
Attention(Q, K, V) = softmax(Q·K^T / √d_k) · V
```

**Componentes:**
- **Q (Query)**: "¿Qué estoy buscando?"
- **K (Key)**: "¿Qué información tengo?"
- **V (Value)**: "La información que proporciono"

In [ ]:
# Ejemplo simple: Calcular atención manualmente

# Embeddings de ejemplo para 3 palabras
# "El gato duerme"
np.random.seed(42)
d_model = 4
seq_len = 3

X = np.array([
    [1.0, 0.5, 0.2, 0.1],  # "El"
    [0.2, 1.0, 0.3, 0.5],  # "gato"
    [0.3, 0.4, 1.0, 0.2]   # "duerme"
])

print("Entrada X (embeddings):")
print(X)
print(f"Shape: {X.shape} (seq_len={seq_len}, d_model={d_model})")

In [ ]:
# Paso 1: Crear matrices W_Q, W_K, W_V (simplificadas como identidad)
d_k = d_model
W_Q = np.eye(d_model)
W_K = np.eye(d_model)
W_V = np.eye(d_model)

# Paso 2: Proyectar a Q, K, V
Q = X @ W_Q
K = X @ W_K
V = X @ W_V

print("Query Q:")
print(Q)
print("\nKey K:")
print(K)
print("\nValue V:")
print(V)

In [ ]:
# Paso 3: Calcular scores de atención (Q·K^T)
scores = Q @ K.T

print("Scores (Q·K^T):")
print(scores)
print(f"\nInterpretación: scores[i,j] = compatibilidad entre palabra i y palabra j")
print(f"scores[0,1] = {scores[0,1]:.3f} → compatibilidad 'El' con 'gato'")
print(f"scores[1,2] = {scores[1,2]:.3f} → compatibilidad 'gato' con 'duerme'")

In [ ]:
# Paso 4: Escalar por √d_k
scaled_scores = scores / np.sqrt(d_k)

print(f"Scores escalados (dividir por √{d_k} = {np.sqrt(d_k):.2f}):")
print(scaled_scores)
print(f"\n¿Por qué escalar? Para mantener varianza estable y gradientes saludables")

In [ ]:
# Paso 5: Aplicar softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

attention_weights = softmax(scaled_scores)

print("Pesos de atención (después de softmax):")
print(attention_weights)
print(f"\nVerificación: Cada fila suma 1.0")
print(f"Suma fila 0: {attention_weights[0].sum():.4f}")
print(f"Suma fila 1: {attention_weights[1].sum():.4f}")
print(f"Suma fila 2: {attention_weights[2].sum():.4f}")

In [ ]:
# Paso 6: Aplicar atención a valores
output = attention_weights @ V

print("Output final:")
print(output)
print(f"\nShape: {output.shape}")
print(f"\nInterpretación:")
print(f"output[0] = representación contextual de 'El'")
print(f"         = {attention_weights[0,0]:.3f} * V[0] + {attention_weights[0,1]:.3f} * V[1] + {attention_weights[0,2]:.3f} * V[2]")

In [ ]:
# Visualizar matriz de atención
tokens = ['El', 'gato', 'duerme']
fig = visualize_attention_weights(attention_weights, tokens, "Self-Attention: Paso a Paso")
plt.show()

print("\n📊 Interpretación de la matriz:")
print("- Filas: Palabras que atienden (Queries)")
print("- Columnas: Palabras atendidas (Keys)")
print("- Valores: Cuánta atención se da")

### 1.2 Self-Attention con Nuestra Implementación

In [ ]:
# Usar nuestra clase SelfAttentionNumPy
np.random.seed(42)

# Secuencia más larga
seq_len, d_model = 7, 16
X = np.random.randn(seq_len, d_model) * 0.5

# Crear self-attention
attention = SelfAttentionNumPy(d_model, d_k=16, seed=42)

# Forward pass
output, attn_weights = attention(X, return_attention=True)

print(f"Entrada: {X.shape}")
print(f"Salida: {output.shape}")
print(f"Pesos de atención: {attn_weights.shape}")

# Visualizar
tokens = ['El', 'perro', 'negro', 'corrió', 'por', 'el', 'parque']
fig = visualize_attention_weights(attn_weights, tokens, "Self-Attention: Ejemplo Completo")
plt.show()

### 🎯 Ejercicio 1: Self-Attention con Máscara Causal

Implementa una máscara causal para prevenir que las palabras atiendan al futuro (necesario en GPT).

In [ ]:
# Ejercicio 1: Máscara Causal

# TODO: Crear máscara causal
causal_mask = create_causal_mask(seq_len)

print("Máscara causal:")
print(causal_mask.astype(int))
print("\nTrue = posición enmascarada (no puede atender)")

# TODO: Aplicar self-attention con máscara
output_masked, attn_masked = attention(X, mask=causal_mask, return_attention=True)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sin máscara
im1 = axes[0].imshow(attn_weights, cmap='viridis', aspect='auto')
axes[0].set_title('Sin máscara (bidireccional)')
axes[0].set_xlabel('Key')
axes[0].set_ylabel('Query')
plt.colorbar(im1, ax=axes[0])

# Con máscara
im2 = axes[1].imshow(attn_masked, cmap='viridis', aspect='auto')
axes[1].set_title('Con máscara causal (GPT-style)')
axes[1].set_xlabel('Key')
axes[1].set_ylabel('Query')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

print("\n📌 Observa: Con máscara, cada posición solo atiende a posiciones anteriores")

---
## Parte 2: Multi-Head Attention

### 2.1 ¿Por qué Multi-Head?

- **Múltiples perspectivas**: Diferentes cabezas aprenden diferentes tipos de relaciones
- **Mayor capacidad**: Cada cabeza se especializa
- **Paralelización**: Todas las cabezas se computan simultáneamente

**Ejemplo:**
- Head 1: Relaciones sintácticas (sujeto-verbo)
- Head 2: Relaciones semánticas (palabras relacionadas)
- Head 3: Posiciones relativas
- Head 4: Correferencia (pronombres)

In [ ]:
# Crear Multi-Head Attention
np.random.seed(42)

seq_len, d_model, num_heads = 6, 64, 8
X = np.random.randn(seq_len, d_model) * 0.3

# Multi-head attention
mha = MultiHeadAttentionNumPy(d_model, num_heads, seed=42)

# Forward pass
output, attn_list = mha(X, return_attention=True)

print(f"Entrada: {X.shape}")
print(f"Número de cabezas: {num_heads}")
print(f"Dimensión por cabeza: {d_model // num_heads}")
print(f"Salida: {output.shape}")
print(f"\nAtención por cabeza:")
for i, attn in enumerate(attn_list):
    print(f"  Head {i+1}: {attn.shape}")

In [ ]:
# Visualizar diferentes cabezas
tokens = ['La', 'niña', 'lee', 'un', 'libro', 'interesante']
fig = visualize_multi_head_attention(attn_list, tokens, num_heads_to_show=4)
plt.suptitle('Multi-Head Attention: Diferentes Patrones', fontsize=14, y=1.02)
plt.show()

print("\n🔍 Observación:")
print("Cada cabeza aprende diferentes patrones de atención")
print("Algunas se enfocan en palabras cercanas, otras en relaciones específicas")

### 🎯 Ejercicio 2: Comparar Número de Cabezas

Experimenta con diferente número de cabezas y observa los patrones.

In [ ]:
# Ejercicio 2: Comparar diferentes números de cabezas

np.random.seed(42)
X = np.random.randn(6, 64) * 0.3
tokens = ['El', 'gato', 'persigue', 'al', 'ratón', 'rápido']

head_configs = [1, 4, 8, 16]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, num_heads in enumerate(head_configs):
    # TODO: Crear multi-head attention con num_heads
    mha = MultiHeadAttentionNumPy(64, num_heads, seed=42)
    
    # TODO: Calcular atención
    output, attn_list = mha(X, return_attention=True)
    
    # Promediar todas las cabezas
    avg_attn = np.mean(np.array(attn_list), axis=0)
    
    # Visualizar
    im = axes[idx].imshow(avg_attn, cmap='viridis', aspect='auto')
    axes[idx].set_title(f'{num_heads} cabezas')
    axes[idx].set_xticks(range(len(tokens)))
    axes[idx].set_yticks(range(len(tokens)))
    axes[idx].set_xticklabels(tokens, rotation=45, ha='right')
    axes[idx].set_yticklabels(tokens)
    plt.colorbar(im, ax=axes[idx])

plt.tight_layout()
plt.show()

print("\n💡 Pregunta de reflexión:")
print("¿Cómo cambian los patrones con más cabezas?")
print("¿Hay un punto de rendimiento decreciente?")

---
## Parte 3: Positional Encoding

### 3.1 El Problema de la Posición

Self-attention es **permutation-invariant**: no distingue orden.

```
"El gato persigue al ratón" = "ratón al persigue gato El" ❌
```

**Solución:** Añadir información de posición mediante Positional Encoding

In [ ]:
# Crear positional encoding
d_model = 128
max_len = 100

pe = PositionalEncodingSinusoidal(d_model, max_len)

print(f"Positional Encoding creado:")
print(f"  Dimensión del modelo: {d_model}")
print(f"  Longitud máxima: {max_len}")
print(f"  Shape: {pe.encoding.shape}")

In [ ]:
# Visualizar positional encoding
fig = visualize_positional_encoding(d_model=128, max_len=100)
plt.show()

print("\n🎨 Interpretación:")
print("- Diferentes frecuencias en diferentes dimensiones")
print("- Dimensiones bajas: frecuencias altas (cambio rápido)")
print("- Dimensiones altas: frecuencias bajas (cambio lento)")
print("- Patrón único para cada posición")

In [ ]:
# Examinar encoding de posiciones específicas
positions_to_check = [0, 10, 20, 50]

plt.figure(figsize=(12, 6))

for pos in positions_to_check:
    encoding_pos = pe.get_encoding(pos+1)[pos]
    plt.plot(encoding_pos, label=f'pos={pos}', alpha=0.7)

plt.xlabel('Dimensión')
plt.ylabel('Valor de encoding')
plt.title('Positional Encoding para diferentes posiciones')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n✅ Cada posición tiene un patrón único")

### 🎯 Ejercicio 3: Similaridad entre Posiciones

Calcula la similitud coseno entre encodings de diferentes posiciones.

In [ ]:
# Ejercicio 3: Similaridad entre posiciones

def cosine_similarity(a, b):
    """Calcula similitud coseno entre dos vectores."""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# TODO: Obtener encoding para 50 posiciones
seq_len = 50
encodings = pe.get_encoding(seq_len)

# TODO: Calcular matriz de similaridad
similarity_matrix = np.zeros((seq_len, seq_len))
for i in range(seq_len):
    for j in range(seq_len):
        similarity_matrix[i, j] = cosine_similarity(encodings[i], encodings[j])

# Visualizar
plt.figure(figsize=(10, 8))
plt.imshow(similarity_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
plt.colorbar(label='Similitud coseno')
plt.xlabel('Posición')
plt.ylabel('Posición')
plt.title('Similaridad entre Positional Encodings')
plt.show()

print("\n📊 Análisis:")
print(f"Similaridad pos[0] vs pos[1]: {similarity_matrix[0, 1]:.3f}")
print(f"Similaridad pos[0] vs pos[10]: {similarity_matrix[0, 10]:.3f}")
print(f"Similaridad pos[0] vs pos[49]: {similarity_matrix[0, 49]:.3f}")
print("\n💡 Posiciones cercanas tienen mayor similaridad")

---
## Parte 4: Transformer Block Completo (PyTorch)

Ahora construiremos un bloque transformer completo usando PyTorch.

In [ ]:
# Importar PyTorch y nuestras implementaciones
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from transformers import (
        TransformerEncoderBlock, TransformerDecoderBlock,
        PositionwiseFeedForward, TransformerModel
    )
    PYTORCH_AVAILABLE = True
    print("✓ PyTorch importado correctamente")
except ImportError:
    PYTORCH_AVAILABLE = False
    print("⚠ PyTorch no disponible. Esta sección se omitirá.")

In [ ]:
if PYTORCH_AVAILABLE:
    # Crear un Transformer Encoder Block
    d_model = 512
    num_heads = 8
    d_ff = 2048
    dropout = 0.1
    
    encoder_block = TransformerEncoderBlock(d_model, num_heads, d_ff, dropout)
    
    print("Transformer Encoder Block:")
    print(encoder_block)
    
    # Contar parámetros
    num_params = sum(p.numel() for p in encoder_block.parameters())
    print(f"\nNúmero de parámetros: {num_params:,}")

In [ ]:
if PYTORCH_AVAILABLE:
    # Test forward pass
    batch_size = 2
    seq_len = 10
    
    # Input aleatorio
    x = torch.randn(batch_size, seq_len, d_model)
    
    # Forward pass
    output = encoder_block(x)
    
    print(f"Input shape: {x.shape}")
    print(f"Output shape: {output.shape}")
    print(f"\n✓ La dimensión se preserva: (batch, seq_len, d_model)")

### 4.1 Transformer Completo

In [ ]:
if PYTORCH_AVAILABLE:
    # Crear Transformer completo
    vocab_size = 10000
    d_model = 512
    num_heads = 8
    num_layers = 6
    d_ff = 2048
    
    transformer = TransformerModel(
        vocab_size=vocab_size,
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers,
        d_ff=d_ff,
        dropout=0.1
    )
    
    print("Transformer Model (Encoder-Decoder):")
    print(f"  Vocabulario: {vocab_size}")
    print(f"  Dimensión del modelo: {d_model}")
    print(f"  Capas: {num_layers}")
    print(f"  Cabezas de atención: {num_heads}")
    
    # Contar parámetros
    total_params = sum(p.numel() for p in transformer.parameters())
    print(f"\nTotal de parámetros: {total_params:,}")
    print(f"Tamaño aproximado: {total_params * 4 / 1024**2:.2f} MB (float32)")

In [ ]:
if PYTORCH_AVAILABLE:
    # Test forward pass del transformer completo
    batch_size = 2
    src_len = 15
    tgt_len = 12
    
    # Secuencias de entrada (índices de tokens)
    src = torch.randint(0, vocab_size, (batch_size, src_len))
    tgt = torch.randint(0, vocab_size, (batch_size, tgt_len))
    
    # Crear máscara causal para decoder
    tgt_mask = transformer.generate_square_subsequent_mask(tgt_len)
    
    # Forward pass
    logits = transformer(src, tgt, tgt_mask=tgt_mask)
    
    print(f"Source shape: {src.shape}")
    print(f"Target shape: {tgt.shape}")
    print(f"Output logits shape: {logits.shape}")
    print(f"\nOutput: (batch={batch_size}, tgt_len={tgt_len}, vocab_size={vocab_size})")
    print("\n✓ Para cada posición en target, predice distribución sobre vocabulario")

---
## Parte 5: BERT Fine-tuning para Análisis de Sentimiento

Usaremos Hugging Face Transformers para fine-tuning de BERT.

In [ ]:
# Importar Hugging Face Transformers
try:
    from transformers import BERTSentimentClassifier
    HF_AVAILABLE = True
    print("✓ Hugging Face Transformers disponible")
except:
    HF_AVAILABLE = False
    print("⚠ Hugging Face Transformers no disponible")
    print("Instalar con: pip install transformers")

In [ ]:
if HF_AVAILABLE:
    # Crear clasificador de sentimientos con BERT
    print("Cargando BERT pre-entrenado...")
    classifier = BERTSentimentClassifier(
        model_name='bert-base-uncased',
        num_labels=2  # Positivo/Negativo
    )
    print("✓ BERT cargado")

In [ ]:
if HF_AVAILABLE:
    # Datos de ejemplo para fine-tuning
    train_texts = [
        "This movie is absolutely fantastic! I loved it.",
        "Terrible film. Complete waste of time.",
        "Great performances and amazing cinematography.",
        "Boring and predictable. Not recommended.",
        "One of the best movies I've ever seen!",
        "Awful. I couldn't finish watching it.",
        "Brilliant storytelling and excellent acting.",
        "Very disappointing. Expected much more."
    ]
    
    train_labels = [1, 0, 1, 0, 1, 0, 1, 0]  # 1=Positivo, 0=Negativo
    
    print(f"Dataset de entrenamiento: {len(train_texts)} ejemplos")
    print(f"\nEjemplos:")
    for text, label in zip(train_texts[:3], train_labels[:3]):
        sentiment = "Positivo" if label == 1 else "Negativo"
        print(f"  [{sentiment}] {text}")

In [ ]:
if HF_AVAILABLE and PYTORCH_AVAILABLE:
    # Fine-tuning simple
    optimizer = torch.optim.AdamW(classifier.model.parameters(), lr=2e-5)
    
    print("Entrenando...\n")
    num_epochs = 3
    
    for epoch in range(num_epochs):
        # Un paso de entrenamiento
        loss = classifier.train_step(train_texts, train_labels, optimizer)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")
    
    print("\n✓ Fine-tuning completado")

In [ ]:
if HF_AVAILABLE:
    # Evaluar con nuevos textos
    test_texts = [
        "I really enjoyed this film. Highly recommend!",
        "Not good at all. Very disappointing.",
        "Amazing movie with great actors."
    ]
    
    predictions, probabilities = classifier.predict(test_texts)
    
    print("Predicciones:\n")
    for text, pred, probs in zip(test_texts, predictions, probabilities):
        sentiment = "Positivo" if pred == 1 else "Negativo"
        confidence = probs[pred] * 100
        print(f"Texto: {text}")
        print(f"Predicción: {sentiment} (confianza: {confidence:.1f}%)")
        print(f"Probabilidades: Neg={probs[0]:.3f}, Pos={probs[1]:.3f}\n")

### 5.1 Visualizar Atención de BERT

In [ ]:
if HF_AVAILABLE:
    # Extraer pesos de atención
    text = "This movie is absolutely fantastic and amazing!"
    
    # Obtener atención de última capa
    attention = classifier.get_attention_weights(text, layer=-1)
    
    print(f"Atención extraída:")
    print(f"  Shape: {attention.shape}")
    print(f"  (num_heads={attention.shape[0]}, seq_len={attention.shape[1]})")
    
    # Tokenizar para obtener tokens
    tokens = classifier.tokenizer.tokenize(text)
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    
    # Visualizar primera cabeza
    plt.figure(figsize=(10, 8))
    plt.imshow(attention[0], cmap='viridis', aspect='auto')
    plt.colorbar(label='Peso de atención')
    plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
    plt.yticks(range(len(tokens)), tokens)
    plt.xlabel('Key (tokens atendidos)')
    plt.ylabel('Query (tokens que atienden)')
    plt.title('BERT Attention - Head 1, Última Capa')
    plt.tight_layout()
    plt.show()
    
    print("\n🔍 Observación:")
    print("BERT aprende a atender a palabras relevantes para el sentimiento")

---
## Parte 6: GPT-2 Text Generation

Generación de texto usando GPT-2 pre-entrenado.

In [ ]:
if HF_AVAILABLE:
    from transformers import GPT2TextGenerator
    
    print("Cargando GPT-2...")
    generator = GPT2TextGenerator(model_name='gpt2')
    print("✓ GPT-2 cargado")

In [ ]:
if HF_AVAILABLE:
    # Generar texto
    prompt = "Once upon a time in a magical forest,"
    
    print(f"Prompt: {prompt}\n")
    print("Generando...\n")
    
    generated_texts = generator.generate(
        prompt=prompt,
        max_length=100,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
        num_return_sequences=3
    )
    
    for i, text in enumerate(generated_texts, 1):
        print(f"--- Generación {i} ---")
        print(text)
        print()

In [ ]:
if HF_AVAILABLE:
    # Analizar probabilidades del siguiente token
    text = "The capital of France is"
    
    token_probs = generator.get_next_token_probabilities(text, top_k=10)
    
    print(f"Texto: '{text}'")
    print(f"\nTop-10 tokens más probables:\n")
    
    for token, prob in token_probs:
        print(f"  '{token}': {prob*100:.2f}%")

### 🎯 Ejercicio 4: Experimentar con Parámetros de Generación

Prueba diferentes valores de temperatura, top_k y top_p.

In [ ]:
# Ejercicio 4: Experimentar con generación

if HF_AVAILABLE:
    prompt = "Artificial intelligence is"
    
    configs = [
        {'temperature': 0.5, 'top_k': 50, 'name': 'Conservadora (T=0.5)'},
        {'temperature': 1.0, 'top_k': 50, 'name': 'Balanceada (T=1.0)'},
        {'temperature': 1.5, 'top_k': 50, 'name': 'Creativa (T=1.5)'},
    ]
    
    print(f"Prompt: '{prompt}'\n")
    
    for config in configs:
        print(f"\n{'='*60}")
        print(f"{config['name']}")
        print('='*60)
        
        # TODO: Generar texto con configuración específica
        texts = generator.generate(
            prompt=prompt,
            max_length=80,
            temperature=config['temperature'],
            top_k=config['top_k'],
            num_return_sequences=1
        )
        
        print(texts[0])
    
    print("\n💡 Observación:")
    print("- Temperatura baja → Texto más predecible")
    print("- Temperatura alta → Texto más creativo/aleatorio")

---
## Parte 7: Comparación con RNNs/LSTMs

### 7.1 Ventajas de Transformers

In [ ]:
# Comparación de complejidad computacional

import pandas as pd

comparison_data = [
    {
        'Aspecto': 'Complejidad por capa',
        'RNN/LSTM': 'O(n·d²)',
        'Transformer': 'O(n²·d)',
        'Ganador': 'Depende de n vs d'
    },
    {
        'Aspecto': 'Operaciones secuenciales',
        'RNN/LSTM': 'O(n)',
        'Transformer': 'O(1)',
        'Ganador': 'Transformer'
    },
    {
        'Aspecto': 'Camino máximo',
        'RNN/LSTM': 'O(n)',
        'Transformer': 'O(1)',
        'Ganador': 'Transformer'
    },
    {
        'Aspecto': 'Paralelización',
        'RNN/LSTM': 'No',
        'Transformer': 'Sí',
        'Ganador': 'Transformer'
    },
    {
        'Aspecto': 'Dependencias largas',
        'RNN/LSTM': 'Difícil',
        'Transformer': 'Fácil',
        'Ganador': 'Transformer'
    },
    {
        'Aspecto': 'Memoria para inferencia',
        'RNN/LSTM': 'O(d)',
        'Transformer': 'O(n·d)',
        'Ganador': 'RNN/LSTM'
    },
]

df = pd.DataFrame(comparison_data)
print("\n" + "="*70)
print("COMPARACIÓN: RNN/LSTM vs Transformer")
print("="*70 + "\n")
print(df.to_string(index=False))
print("\n" + "="*70)

In [ ]:
# Simular tiempo de procesamiento

def simulate_processing_time(architecture, seq_len, d_model=512):
    """
    Simula tiempo relativo de procesamiento.
    No es tiempo real, solo comparación relativa.
    """
    if architecture == 'RNN':
        # Procesamiento secuencial: O(n)
        return seq_len
    elif architecture == 'Transformer':
        # Procesamiento paralelo pero O(n²)
        return seq_len ** 1.5  # Simplificado

seq_lengths = [10, 20, 50, 100, 200, 500]
rnn_times = [simulate_processing_time('RNN', n) for n in seq_lengths]
transformer_times = [simulate_processing_time('Transformer', n) for n in seq_lengths]

plt.figure(figsize=(10, 6))
plt.plot(seq_lengths, rnn_times, marker='o', label='RNN (secuencial)', linewidth=2)
plt.plot(seq_lengths, transformer_times, marker='s', label='Transformer (paralelo)', linewidth=2)
plt.xlabel('Longitud de secuencia')
plt.ylabel('Tiempo relativo')
plt.title('Tiempo de Procesamiento: RNN vs Transformer')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print("\n📊 Análisis:")
print("- RNN: Crece linealmente pero es secuencial (lento en GPUs)")
print("- Transformer: Crece más rápido pero se puede paralelizar")
print("- En la práctica con GPUs, Transformers son mucho más rápidos")

---
## Parte 8: Ejercicios Finales

### 🎯 Ejercicio 5: Análisis de Atención en Diferentes Capas

Analiza cómo cambian los patrones de atención a través de las capas de BERT.

In [ ]:
# Ejercicio 5: Atención en diferentes capas

if HF_AVAILABLE:
    from transformers import compare_attention_patterns
    
    text = "The quick brown fox jumps over the lazy dog"
    
    # TODO: Visualizar atención en capas 0, 6 y 11
    fig = compare_attention_patterns(text, layers_to_show=[0, 6, 11])
    plt.show()
    
    print("\n💡 Preguntas de reflexión:")
    print("1. ¿Cómo cambian los patrones de atención entre capas?")
    print("2. ¿Las capas tempranas son más sintácticas o semánticas?")
    print("3. ¿Qué patrones observas en la capa final?")

### 🎯 Ejercicio 6: Fine-tuning Personalizado

Crea tu propio dataset y fine-tunea BERT para una tarea específica.

In [ ]:
# Ejercicio 6: Tu propio fine-tuning

if HF_AVAILABLE and PYTORCH_AVAILABLE:
    # TODO: Define tu dataset (por ejemplo, clasificación de spam)
    custom_texts = [
        # Añade tus ejemplos aquí
        "Get rich quick! Click now!",  # Spam
        "Meeting tomorrow at 3pm",      # No spam
        # ... más ejemplos
    ]
    
    custom_labels = [
        1,  # 1 = Spam
        0,  # 0 = No spam
        # ...
    ]
    
    # TODO: Crea un clasificador
    # spam_classifier = BERTSentimentClassifier(...)
    
    # TODO: Entrena
    # ...
    
    # TODO: Evalúa
    # ...
    
    print("\n🎓 Completa este ejercicio con tu propia tarea de clasificación")

---
## Resumen y Conclusiones

### ✅ Lo que aprendimos:

1. **Self-Attention**: Mecanismo fundamental que permite capturar relaciones entre elementos
2. **Multi-Head Attention**: Múltiples perspectivas en paralelo para mayor capacidad
3. **Positional Encoding**: Solución elegante para incorporar información de posición
4. **Transformer Architecture**: Bloques encoder-decoder con residual connections y layer norm
5. **BERT vs GPT**: Encoder-only (comprensión) vs Decoder-only (generación)
6. **Transfer Learning**: Pre-entrenamiento + fine-tuning para tareas específicas
7. **Visualización**: Interpretación de patrones de atención

### 🚀 Ventajas clave de Transformers:

- ✅ Paralelización completa
- ✅ Captura dependencias de largo alcance
- ✅ Escalabilidad a modelos gigantes
- ✅ Versatilidad (NLP, visión, audio, etc.)
- ✅ Transfer learning efectivo

### 📚 Próximos pasos:

1. Experimenta con modelos más grandes (BERT-Large, GPT-3)
2. Prueba Vision Transformers (ViT) para imágenes
3. Explora modelos multimodales (CLIP, Flamingo)
4. Estudia optimizaciones (Flash Attention, sparse attention)
5. Implementa tu propia aplicación con Transformers

### 🎯 Desafíos avanzados:

- Implementa un Transformer desde cero para traducción
- Fine-tunea GPT-2 en tu propio corpus de texto
- Crea un sistema de Q&A usando BERT
- Experimenta con prompt engineering en GPT
- Visualiza e interpreta attention maps en profundidad

---

## 🎉 ¡Felicitaciones!

Has completado el laboratorio de Transformers. Ahora entiendes la arquitectura que ha revolucionado el Deep Learning y que alimenta los modelos más avanzados de IA actuales (ChatGPT, GPT-4, DALL-E, etc.).

**"Attention is All You Need"** - Vaswani et al., 2017

---